In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import redis
import json
from pprint import pprint

import pymongo
from datetime import datetime, timedelta, date
import pandas as pd


import numpy as np
import pymongo
import cx_Oracle
import os
import time
import plotly.express as px
from datetime import datetime, timedelta, date


pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.options.mode.chained_assignment = None  # default='warn'

r = redis.Redis(host="127.0.0.1", port=6379, db=0)

In [2]:
def encode(s):
    
    i = 0
    t = ''
    while i < len(s):
        if s[i].isupper():
            t += 'U'
        elif s[i].islower():
            t += 'L'
        elif s[i].isdigit():
            t += 'D'
        else:
            t += 'O'
        i += 1
        
    return t

def short_encode(s):
    i = 0
    s = encode(s)
    curr = ''
    t = ''
    
    while i < len(s):
        if curr != s[i]:
            t += s[i]
            curr = s[i]
        
        i += 1
    return t

def score_sus_email(s):

    email_input = s
    email_input = email_input.split('@')[0]
    
    
    
    pat_count_digit = encode(email_input).count('D')
    pat_encode = short_encode(email_input)
    
    if pat_encode == 'LD' and pat_count_digit >= 4:
        return 1
    elif pat_encode == 'D' and pat_count_digit >= 4:
        return 1
    return 0
    
score_sus_email('qywyeqeyqw123@gmail.com')

0

In [3]:
from gibberish_detector import detector
Detector = detector.create_from_model(
    '/home/server/gli-data-science/akhiyar/alfagift_alert/gibberish-detector/big.model')
from nostril import nonsense


def score_gibberish(input_email):
    split_email = input_email.split('@')[0]
    #print(split_email)

    flag_gibberish1 = False
    
    try:
        if Detector.is_gibberish(split_email):
            flag_gibberish1 = True
    except Exception as e:
        pass
    
    flag_gibberish2 = False
    
    try:
        if nonsense(split_email):
            flag_gibberish2 = True
    except Exception as e:
        pass


    if flag_gibberish1 and flag_gibberish2:
        return 1
    else:
        return 0

email = 'puji01236@gmail.com'
score_gibberish(email)

0

In [4]:
alfagift_member = pd.read_json('/home/server/mongo_export/alfagift_member.json')
alfagift_member['sapa'] = alfagift_member['sapa'].fillna('-')
alfagift_member['sapa'] = alfagift_member['sapa'].apply(lambda x: x.get('store_code') \
                                                        if x != '-' else '-')
alfagift_member['sapa'] = alfagift_member['sapa'].apply(lambda x: '-' \
                                                        if x == '' else x)

alfagift_member['created_at'] = alfagift_member['created_at'].fillna('2019-01-01T23:08:53.393Z')
alfagift_member['created_at'] = alfagift_member['created_at'].apply(lambda x: x.get('$date') \
                                                                    if type(x) != str else x)

# alfagift_member = alfagift_member[['account_card', 'sapa', 'created_at']]
alfagift_member['sapa'] = alfagift_member['sapa'].apply(lambda x: 'sapa' if x != '-' else 'not_sapa')

alfagift_member['created_at_d'] = pd.to_datetime(alfagift_member['created_at']).dt.strftime("%Y-%m-%d")
alfagift_member['created_at_m'] = pd.to_datetime(alfagift_member['created_at']).dt.strftime("%Y-%m")

alfagift_member['account_card'] = alfagift_member['account_card'].astype('str')

In [5]:
alfagift_member_sel = alfagift_member[(alfagift_member['created_at_m'] == '2022-01') |
                                      (alfagift_member['created_at_m'] == '2022-02') |
                                      (alfagift_member['created_at_m'] == '2022-03') |
                                      (alfagift_member['created_at_m'] == '2022-04') ]
alfagift_member_sel['created_at_d'] = pd.to_datetime(alfagift_member_sel['created_at']).dt.strftime("%Y-%m-%d")

In [6]:
## creating label for training set




li_sus = []
for dr in pd.date_range('2022-03-01', '2022-04-06', freq='D'):
    sel_created = dr.strftime("%Y-%m-%d")

    alfagift_member_sel_d = alfagift_member_sel[alfagift_member_sel['created_at_d'] == sel_created]
    print("{} {}".format(sel_created, alfagift_member_sel_d.shape))


    ## suspect by IP
    alfagift_member_sel_d_ip = alfagift_member_sel_d.groupby(['createdFromIp'])\
                            .agg(createdFromIp_count=('createdFromIp','count'))
    alfagift_member_sel_d_ip = alfagift_member_sel_d_ip[
        alfagift_member_sel_d_ip['createdFromIp_count'] >= 5].reset_index()

    alfagift_member_sel_d = pd.merge(alfagift_member_sel_d, 
                                     alfagift_member_sel_d_ip,
                                     on='createdFromIp', how='left').fillna(0)

    ## suspect by phone
    alfagift_member_sel_d['phone_sub'] =  alfagift_member_sel_d['phone'].str[0:9]
    alfagift_member_sel_d_phone = alfagift_member_sel_d.groupby(['phone_sub'])\
                                .agg(phoneSub_count=('phone_sub','count'))
    alfagift_member_sel_d_phone = alfagift_member_sel_d_phone[
        alfagift_member_sel_d_phone['phoneSub_count'] >= 5].reset_index()

    
    
    alfagift_member_sel_d = pd.merge(alfagift_member_sel_d, 
                                     alfagift_member_sel_d_phone,
                                     on='phone_sub', how='left').fillna(0)
    
    
    alfagift_member_sel_d['score_gibberish'] = alfagift_member_sel_d['email'].apply(score_gibberish)
    alfagift_member_sel_d['score_sus_email'] = alfagift_member_sel_d['email'].apply(score_sus_email)
    

    li_sus.append(alfagift_member_sel_d)
#     alfagift_member_sel_d.to_csv(
#     '/home/server/gli-data-science/akhiyar/alfagift_alert/suspect_fraud/sus_{}'.format(sel_created))

    #break

df_sus_all = pd.concat(li_sus)
df_sus_all.to_csv(
    '/home/server/gli-data-science/akhiyar/alfagift_alert/suspect_fraud_feature/group_day/train_all.csv'
    , index=False)

2022-03-01 (15581, 14)
2022-03-02 (15078, 14)
2022-03-03 (14881, 14)
2022-03-04 (14451, 14)
2022-03-05 (14248, 14)
2022-03-06 (14713, 14)
2022-03-07 (15282, 14)
2022-03-08 (15663, 14)
2022-03-09 (16846, 14)
2022-03-10 (19367, 14)
2022-03-11 (33149, 14)
2022-03-12 (17645, 14)
2022-03-13 (15917, 14)
2022-03-14 (16535, 14)
2022-03-15 (16187, 14)
2022-03-16 (18047, 14)
2022-03-17 (17248, 14)
2022-03-18 (16520, 14)
2022-03-19 (16641, 14)
2022-03-20 (15306, 14)
2022-03-21 (16749, 14)
2022-03-22 (17836, 14)
2022-03-23 (18728, 14)
2022-03-24 (22499, 14)
2022-03-25 (32843, 14)
2022-03-26 (20540, 14)
2022-03-27 (15542, 14)
2022-03-28 (15255, 14)
2022-03-29 (14883, 14)
2022-03-30 (15741, 14)
2022-03-31 (13978, 14)
2022-04-01 (13748, 14)
2022-04-02 (14113, 14)
2022-04-03 (13247, 14)
2022-04-04 (11072, 14)
2022-04-05 (11260, 14)
2022-04-06 (10798, 14)


In [16]:
df_label = pd.read_csv('./suspect_fraud/sus_all.csv', dtype='object')
df_label['label'] = 1

In [17]:
df_train = pd.merge(df_sus_all, df_label, on='account_card', how='left')

In [19]:
df_train['label'] = df_train['label'].fillna(0)

In [20]:
df_train.head()

,_id,member_status,full_name,gender,email,birth_date,phone,marital_status,account_card,created_at,sapa,createdFromIp,created_at_d,created_at_m,createdFromIp_count,phone_sub,phoneSub_count,score_gibberish,score_sus_email,label
0,{'$oid': '621d6204112eab3ec42a7c9c'},26,prasetya,M,setyaprasgaplek@gmail.com,{'$date': '1995-06-30T00:00:00Z'},082167876251,S,9990013684045752,2022-03-01T00:00:04.466Z,not_sapa,114.79.16.192,2022-03-01,2022-03,0.00,082167876,0.00,0,0,0.00
1,{'$oid': '621d621521cf444cff0797b1'},26,septian,M,kknterboyowetan2017@gmail.com,{'$date': '1994-03-01T00:00:00Z'},085740644763,M,9990013684045866,2022-03-01T00:00:21.886Z,not_sapa,114.10.7.31,2022-03-01,2022-03,0.00,085740644,0.00,0,1,0.00
2,{'$oid': '621d622767177320465503a3'},26,annang priyo jadmiko,M,annanginung@gmail.com,{'$date': '1992-10-09T00:00:00Z'},087755509450,S,9990013684045980,2022-03-01T00:00:39.092Z,not_sapa,103.36.10.162,2022-03-01,2022-03,0.00,087755509,0.00,0,0,0.00
3,{'$oid': '621d622d112eab3ec42a7ca2'},26,MUCHAMAD ARYA ARIFIN,M,muchamadaryaarifinkias@gmail.com,{'$date': '2000-05-10T00:00:00Z'},085641143095,S,9990013681564332,2022-03-01T00:00:45.359Z,not_sapa,125.163.45.193,2022-03-01,2022-03,0.00,085641143,0.00,0,0,0.00
4,{'$oid': '621d6240fb14cd5a8baa9190'},26,Hayatun Nupus,F,nufussunu2901@gmail.com,{'$date': '2000-07-29T00:00:00Z'},087781270536,S,9990013684046067,2022-03-01T00:01:04.216Z,not_sapa,114.122.169.183,2022-03-01,2022-03,0.00,087781270,0.00,0,1,0.00


In [24]:
wrap_col = [
'score_gibberish',
'score_sus_email',
'createdFromIp_count',
'phoneSub_count',

]
X = df_train[wrap_col]
y = df_train['label']

In [25]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_text
regressor = DecisionTreeClassifier(random_state=0, max_depth=4)
regressor.fit(X,y)
r = export_text(regressor, feature_names=list(X), decimals=0, spacing=6)
print("prediction rule \n\n{}".format(r))

prediction rule 

|------ createdFromIp_count <= 2
|      |------ phoneSub_count <= 2
|      |      |------ class: 0.0
|      |------ phoneSub_count >  2
|      |      |------ class: 1.0
|------ createdFromIp_count >  2
|      |------ class: 1.0



In [26]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0)
clf.fit(X, y)

LogisticRegression(random_state=0)

In [42]:
pd.concat([pd.DataFrame(list(X), columns=['variabel']), 
           pd.DataFrame(pd.Series(clf.coef_[0]), columns=['bobot'])], 1)\
.append({'variabel':'intercept','bobot':clf.intercept_[0]},ignore_index=True)

,variabel,bobot
0,score_gibberish,0.17
1,score_sus_email,0.06
2,createdFromIp_count,4.30
3,phoneSub_count,4.26
4,intercept,-12.71
